# 01 — Data Collection

**Goal:** Collect VIX and S&P 500 daily data from Yahoo Finance using Playwright and BeautifulSoup.

**Outputs:**
- `data/raw/vix_ytd_ohlc.csv`
- `data/raw/sp500_ytd_ohlcv.csv`

**Notes:**  
- Only educational use (respect Yahoo ToS).  
- Dates in UTC; daily frequency.  
- The scraping code lives in `/scripts/`.